# SMS Spam Filter With Naive Bayes Algorithm From Scratch Using R

## Introduction

In this project, I am going to build a SMS spam filter in R with Naive Bayes algorithm. To implement the Naive Bayes algorithm, you may use the `naiveBayes` function included in the `e1071` package, which is a very famous and widely-used R package for maching learning. However, to develop deeper understanding of the statistical theory behind the algorithm, my goal is to build the SMS spam filter from scratch rather than just implementing the alogirthm from the package.

The dataset for this project can be downloaded from [Kaggle](https://www.kaggle.com/uciml/sms-spam-collection-dataset/data). This dataset includes 5572 SMS messages, labelled with either ham or spam.

I will start by setting up a workplace, importing the required packages and loading the dataset.

In [1]:
# Clean working environment
rm(list=ls())

# Import packages
library(tm)
library(wordcloud)
library(ggplot2)
library(repr)
library(rlist)
library(gmodels)

# Load dataset
sms <- read.csv("datasets_483_982_spam.csv", stringsAsFactors=FALSE, encoding="latin1")

# Show dataset structure
str(sms)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Loading required package: NLP
Loading required package: RColorBrewer

Attaching package: ‘ggplot2’

The following object is masked from ‘package:NLP’:

    annotate



'data.frame':	5572 obs. of  5 variables:
 $ v1 : chr  "ham" "ham" "spam" "ham" ...
 $ v2 : chr  "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..." "Ok lar... Joking wif u oni..." "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question("| __truncated__ "U dun say so early hor... U c already then say..." ...
 $ X  : chr  "" "" "" "" ...
 $ X.1: chr  "" "" "" "" ...
 $ X.2: chr  "" "" "" "" ...


## Data Cleaning

The above result shows that the `sms` data frame contains five features. However, only the first two features are essential of constructing a SMS spam filter. Moreover, the text in `sms` contains a lot of redundant information, like puncuation, whiterspace and numbers. Hence, in this section, all the unusable information will be removed.

In [2]:
# Remove column 3,4,5
sms_clean <- sms[,-(3:5)]
colnames(sms_clean) <- c("label", "text")

# Remove redundant information
corpus <- Corpus(VectorSource(sms_clean$text))
corpus_clean <- tm_map(corpus, tolower)
corpus_clean <- tm_map(corpus_clean, removeNumbers)
corpus_clean <- tm_map(corpus_clean, removeWords, stopwords())
corpus_clean <- tm_map(corpus_clean, removePunctuation)
corpus_clean <- tm_map(corpus_clean, stripWhitespace)

# Transform to a document-term matrix
sms_dtm <- as.matrix(DocumentTermMatrix(corpus_clean))

Warning message in tm_map.SimpleCorpus(corpus, tolower):
“transformation drops documents”Warning message in tm_map.SimpleCorpus(corpus_clean, removeNumbers):
“transformation drops documents”Warning message in tm_map.SimpleCorpus(corpus_clean, removeWords, stopwords()):
“transformation drops documents”Warning message in tm_map.SimpleCorpus(corpus_clean, removePunctuation):
“transformation drops documents”Warning message in tm_map.SimpleCorpus(corpus_clean, stripWhitespace):
“transformation drops documents”

## Data Exploration

To get richer understanding to the data, I'll explore the dataset before proceeding to some deeper analysis.

In [3]:
# Show first few row of dataset
head(sms_clean)

label,text
<chr>,<chr>
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives around here though"
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"


In [4]:
# Show ratio of ham and spam
round(table(sms_clean$label)/length(sms_clean$label), 3)

# Visulise no. of spam and ham
options(repr.plot.width=3, repr.plot.height=4.5)
ggplot(sms_clean, aes(x=label)) + geom_bar(fill="seagreen1", width=0.5) + xlab("SMS type") + ylab("No. of observation") + theme(axis.line=element_line(color="black"))


  ham  spam 
0.866 0.134 

In [5]:
# Show the most common words in SMS
word_cnt <- sort(apply(sms_dtm, 2, sum), decreasing=TRUE)
word_cnt_df <- data.frame(word=names(word_cnt), freq.=word_cnt)
rownames(word_cnt_df) <- NULL
head(word_cnt_df)

# Visualise words
options(repr.plot.width=9.375, repr.plot.height=6.25)
wordcloud(words=names(word_cnt), freq=word_cnt, min.freq=50, max.words=100, rot.per=0.3, colors = brewer.pal(5, "Dark2"))

word,freq.
<fct>,<dbl>
call,579
now,483
can,405
get,390
will,380
just,367


## Creating Custom Function for Naive Bayes Classifier

At the beginning of this article, I mentioned that you may use the `e1071` package to implement the Naive Bayes algorithm. It includes a bunch of function for machine learning. You can build the classifier with `naiveBayes` function in `e1071`.

Another way to perform model trainning and classification is using the custom function I wrote.

The `NaiveBayes` function create a list of storing information like prior probability and likelihood. To learn more about Bayesian statistics, you can visit https://en.wikipedia.org/wiki/Bayesian_statistics.

The `NaiveBayes` function takes the following arguments:
* **dtm** - The document-term matrix  
* **type** - The label vector  
* **laplace** - The number controlling laplace smoothing

Let's see the code given below.

In [6]:
# Create function to build classifier
NaiveBayes <- function(dtm, type, laplace=0){
    words <- colnames(dtm)
    dtm <- as.data.frame(dtm)
    dtm <- cbind(dtm, type)
    
    names <- unique(type); dtm_type<- list()
    for(i in 1:length(names)){
        dtm_type <- list.append(dtm_type, dtm[dtm[,ncol(dtm)]==names[i],-ncol(dtm)])
    }
    names(dtm_type) <- names
    
    word_cnt <- list()
    for(i in 1:length(dtm_type)){
        cnt <- apply(dtm_type[[i]], 2, sum)
        names(cnt) <- words
        word_cnt <- list.append(word_cnt, cnt)
    }
    names(word_cnt) <- names
    
    prob <- list()
    for(i in 1:length(word_cnt)){
        p <- (word_cnt[[i]]+laplace)/sum(word_cnt[[i]]+laplace)
        names(p) <- words
        prob <- list.append(prob, p)
    }
    names(prob) <- names
    
    prior <- list()
    for(i in 1:length(dtm_type)){
        prior <- list.append(prior, nrow(dtm_type[[i]])/length(type))
    }
    names(prior) <- names
    
    result <- list()
    result <- list.append(result, Words=words)
    result <- list.append(result, Types=names)
    result <- list.append(result, PriorProbability=prior)
    result <- list.append(result, Likelihood=prob)
    return(result)
}

The `NaiveBayesClassifier` function is to classify whether or not the SMS is spam.

The `NaiveBayesClassifier` function takes the following arguments:
* **test** - The vector to be test
* **classifier** - The model trainned by `NaiveBayes` function

In [7]:
# Create function to classify whether SMS is spam
NaiveBayesClassifier <- function(test, classifier){
    test_word <- test[test %in% classifier$Words]
    fre <- table(test_word);
    
    posterior <- NULL; prob <- classifier$Likelihood
    for(i in 1:length(prob)){
        posterior[i] <- prod(prob[[i]][names(prob[[i]]) %in% names(fre)]^fre) * classifier$PriorProbability[[i]]
    }

    names(posterior) <- classifier$Types
    result <- list()
    result <- list.append(result, round(posterior, 5))
    result <- list.append(result, names(posterior[posterior == max(posterior)]))
    return(result[[2]])
}

## Model Trainning

Now I'll split the data into trainning and testing dataset before performing model trainning. As soon as the model had been trained, the testing dataset can be used to evaluate the model performance.

In [8]:
# Split Data
split_pt <- round(nrow(sms_dtm)*0.75,0)

# Data for trainning
train <- sms_dtm[1:split_pt,]
type <- sms_clean[1:split_pt,1]

# Data for testing
test <- corpus_clean$content[(split_pt+1):length(corpus_clean$content)]
test <- strsplit(test," ")

Use the custom function `NaiveBayes` to build the spam filter.

In [9]:
# Create spam filter
filter <- NaiveBayes(train, type, laplace=1)

Now the `filter` variable can be used to make some predictions on the testing dataset.

## Evaluating Model Performance

Once the model had been trained, we need to evaluate its performance before using the model to make prediction.

In [10]:
# Predict testing dataset
result <- 0
for(i in 1:length(test)){
    t <- unlist(test[i])
    result[i] <- NaiveBayesClassifier(t, filter)
}

# Evaluate performance
real <- sms_clean[(split_pt+1):nrow(sms_clean), 1]
CrossTable(result, real, prop.chisq=F, prop.t=F, dnn=c("predict","actual"))


 
   Cell Contents
|-------------------------|
|                       N |
|           N / Row Total |
|           N / Col Total |
|-------------------------|

 
Total Observations in Table:  1393 

 
             | actual 
     predict |       ham |      spam | Row Total | 
-------------|-----------|-----------|-----------|
         ham |      1181 |        12 |      1193 | 
             |     0.990 |     0.010 |     0.856 | 
             |     0.975 |     0.066 |           | 
-------------|-----------|-----------|-----------|
        spam |        30 |       170 |       200 | 
             |     0.150 |     0.850 |     0.144 | 
             |     0.025 |     0.934 |           | 
-------------|-----------|-----------|-----------|
Column Total |      1211 |       182 |      1393 | 
             |     0.869 |     0.131 |           | 
-------------|-----------|-----------|-----------|

 


The crosstab shows that 42 out of 1393 SMS were incorrectly classified. In other words only 3% of them cannot assign to its own class. So, in my opinion, the model perform pretty well.